In [104]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV
import imblearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [48]:
from google.colab import drive

In [49]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
auto_train_ds = pd.read_csv("/content/drive/MyDrive/Main_Project007/Train.csv", sep=",")
auto_test_ds = pd.read_csv("/content/drive/MyDrive/Main_Project007/Test.csv", sep=",")
data = pd.concat([auto_train_ds, auto_test_ds])

data

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A
...,...,...,...,...,...,...,...,...,...,...,...
2622,467954,Male,No,29,No,Healthcare,9.0,Low,4.0,Cat_6,B
2623,467958,Female,No,35,Yes,Doctor,1.0,Low,1.0,Cat_6,A
2624,467960,Female,No,53,Yes,Entertainment,NaN,Low,2.0,Cat_6,C
2625,467961,Male,Yes,47,Yes,Executive,1.0,High,5.0,Cat_4,C


In [51]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10695 entries, 0 to 2626
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               10695 non-null  int64  
 1   Gender           10695 non-null  object 
 2   Ever_Married     10505 non-null  object 
 3   Age              10695 non-null  int64  
 4   Graduated        10593 non-null  object 
 5   Profession       10533 non-null  object 
 6   Work_Experience  9597 non-null   float64
 7   Spending_Score   10695 non-null  object 
 8   Family_Size      10247 non-null  float64
 9   Var_1            10587 non-null  object 
 10  Segmentation     10695 non-null  object 
dtypes: float64(2), int64(2), object(7)
memory usage: 1002.7+ KB


In [52]:
data.isnull().sum()

ID                    0
Gender                0
Ever_Married        190
Age                   0
Graduated           102
Profession          162
Work_Experience    1098
Spending_Score        0
Family_Size         448
Var_1               108
Segmentation          0
dtype: int64

In [175]:


enc=OneHotEncoder()
enc_data=pd.DataFrame(enc.fit_transform(data[['segment_new']]).toarray())
 
#Merge with main
data=data.join(enc_data)

In [53]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10695 entries, 0 to 2626
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               10695 non-null  int64  
 1   Gender           10695 non-null  object 
 2   Ever_Married     10505 non-null  object 
 3   Age              10695 non-null  int64  
 4   Graduated        10593 non-null  object 
 5   Profession       10533 non-null  object 
 6   Work_Experience  9597 non-null   float64
 7   Spending_Score   10695 non-null  object 
 8   Family_Size      10247 non-null  float64
 9   Var_1            10587 non-null  object 
 10  Segmentation     10695 non-null  object 
dtypes: float64(2), int64(2), object(7)
memory usage: 1002.7+ KB


In [54]:
data['Work_Experience'] = data['Work_Experience'].fillna(data['Work_Experience'].mean())
data['Family_Size'] = data['Family_Size'].fillna(data['Family_Size'].mean())

In [55]:
data['Ever_Married'] = data['Ever_Married'].fillna(data['Ever_Married'].mode()[0])
data['Graduated'] = data['Graduated'].fillna(data['Graduated'].mode()[0])
data['Profession'] = data['Profession'].fillna(data['Profession'].mode()[0])

In [56]:
data.isnull().sum()
data

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.000000,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,2.619777,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.000000,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.000000,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,2.619777,High,6.0,Cat_6,A
...,...,...,...,...,...,...,...,...,...,...,...
2622,467954,Male,No,29,No,Healthcare,9.000000,Low,4.0,Cat_6,B
2623,467958,Female,No,35,Yes,Doctor,1.000000,Low,1.0,Cat_6,A
2624,467960,Female,No,53,Yes,Entertainment,2.619777,Low,2.0,Cat_6,C
2625,467961,Male,Yes,47,Yes,Executive,1.000000,High,5.0,Cat_4,C


In [57]:
cols = ['ID', 'Var_1']
data = data.drop(columns=cols, axis=1)

In [58]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10695 entries, 0 to 2626
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Gender           10695 non-null  object 
 1   Ever_Married     10695 non-null  object 
 2   Age              10695 non-null  int64  
 3   Graduated        10695 non-null  object 
 4   Profession       10695 non-null  object 
 5   Work_Experience  10695 non-null  float64
 6   Spending_Score   10695 non-null  object 
 7   Family_Size      10695 non-null  float64
 8   Segmentation     10695 non-null  object 
dtypes: float64(2), int64(1), object(6)
memory usage: 835.5+ KB


In [59]:
t1 = pd.get_dummies(data['Gender'], drop_first= True)
t2 = pd.get_dummies(data['Ever_Married'], drop_first= True)
t3 = pd.get_dummies(data['Graduated'], drop_first= True)
t4 = pd.get_dummies(data['Profession'], drop_first= True)
t5 = pd.get_dummies(data['Spending_Score'], drop_first= True)





data = pd.concat([data, t1, t2, t3, t4, t5], axis = 1)

cols = ['Gender', 'Ever_Married', "Graduated", "Profession", "Spending_Score"]
data = data.drop(columns=cols, axis=1)

In [60]:
data["Age_Range"] = np.where(data.Age>=60, "60+", np.where(data.Age>=50, "50-60", np.where(data.Age>=40, "40-50", np.where(data.Age>=30, "30-40", np.where(data.Age>=18, "18-30", "18-")))))

In [61]:
d1=pd.get_dummies(data['Age_Range'], drop_first= True)
data = pd.concat([data,d1], axis = 1)

In [62]:
data.head()

,Age,Work_Experience,Family_Size,Segmentation,Male,Yes,Yes,Doctor,Engineer,Entertainment,...,Homemaker,Lawyer,Marketing,High,Low,Age_Range,30-40,40-50,50-60,60+
0,22,1.000000,4.0,D,1,0,0,0,0,0,...,0,0,0,0,1,18-30,0,0,0,0
1,38,2.619777,3.0,A,0,1,1,0,1,0,...,0,0,0,0,0,30-40,1,0,0,0
2,67,1.000000,1.0,B,0,1,1,0,1,0,...,0,0,0,0,1,60+,0,0,0,1
3,67,0.000000,2.0,B,1,1,1,0,0,0,...,0,1,0,1,0,60+,0,0,0,1
4,40,2.619777,6.0,A,0,1,1,0,0,1,...,0,0,0,1,0,40-50,0,1,0,0


In [63]:
data = data.drop(['Age'], axis=1)

In [64]:
data["Work_Experience_Range"] = np.where(data.Work_Experience>=10, "10+", np.where(data.Work_Experience>=5, "5-10", "0-5"))
data["Family_Size_Range"] = np.where(data.Family_Size>=6, "6+", np.where(data.Family_Size>=3, "3-6", "0-3"))

In [65]:
data = data.drop(['Work_Experience','Family_Size'], axis=1)

In [66]:
d1=pd.get_dummies(data['Work_Experience_Range'], drop_first= True)
d2=pd.get_dummies(data['Family_Size_Range'], drop_first= True)
data = pd.concat([data,d1,d2], axis = 1)
data.head()

,Segmentation,Male,Yes,Yes,Doctor,Engineer,Entertainment,Executive,Healthcare,Homemaker,...,30-40,40-50,50-60,60+,Work_Experience_Range,Family_Size_Range,10+,5-10,3-6,6+
0,D,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0-5,3-6,0,0,1,0
1,A,0,1,1,0,1,0,0,0,0,...,1,0,0,0,0-5,3-6,0,0,1,0
2,B,0,1,1,0,1,0,0,0,0,...,0,0,0,1,0-5,0-3,0,0,0,0
3,B,1,1,1,0,0,0,0,0,0,...,0,0,0,1,0-5,0-3,0,0,0,0
4,A,0,1,1,0,0,1,0,0,0,...,0,1,0,0,0-5,6+,0,0,0,1


In [67]:
data = data.drop(['Age_Range','Work_Experience_Range','Family_Size_Range'], axis=1)

In [68]:
data.head()

,Segmentation,Male,Yes,Yes,Doctor,Engineer,Entertainment,Executive,Healthcare,Homemaker,...,High,Low,30-40,40-50,50-60,60+,10+,5-10,3-6,6+
0,D,1,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0
1,A,0,1,1,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
2,B,0,1,1,0,1,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,B,1,1,1,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
4,A,0,1,1,0,0,1,0,0,0,...,1,0,0,1,0,0,0,0,0,1


In [69]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10695 entries, 0 to 2626
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Segmentation   10695 non-null  object
 1   Male           10695 non-null  uint8 
 2   Yes            10695 non-null  uint8 
 3   Yes            10695 non-null  uint8 
 4   Doctor         10695 non-null  uint8 
 5   Engineer       10695 non-null  uint8 
 6   Entertainment  10695 non-null  uint8 
 7   Executive      10695 non-null  uint8 
 8   Healthcare     10695 non-null  uint8 
 9   Homemaker      10695 non-null  uint8 
 10  Lawyer         10695 non-null  uint8 
 11  Marketing      10695 non-null  uint8 
 12  High           10695 non-null  uint8 
 13  Low            10695 non-null  uint8 
 14  30-40          10695 non-null  uint8 
 15  40-50          10695 non-null  uint8 
 16  50-60          10695 non-null  uint8 
 17  60+            10695 non-null  uint8 
 18  10+            10695 non-nu

In [70]:
data.head()

,Segmentation,Male,Yes,Yes,Doctor,Engineer,Entertainment,Executive,Healthcare,Homemaker,...,High,Low,30-40,40-50,50-60,60+,10+,5-10,3-6,6+
0,D,1,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0
1,A,0,1,1,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
2,B,0,1,1,0,1,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,B,1,1,1,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
4,A,0,1,1,0,0,1,0,0,0,...,1,0,0,1,0,0,0,0,0,1


In [71]:
x = data.drop(columns=['Segmentation'], axis=1)
y = data['Segmentation']

In [123]:
data.drop([],axis=1)

,segment_new,0,2,3,Male,Yes,Yes,Doctor,Engineer,Entertainment,...,High,Low,30-40,40-50,50-60,60+,10+,5-10,3-6,6+
0,3,0.0,0.0,1.0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
0,1,0.0,0.0,1.0,0,1,1,0,1,0,...,0,1,1,0,0,0,0,0,0,0
1,0,1.0,0.0,0.0,0,1,1,0,1,0,...,0,0,1,0,0,0,0,0,1,0
1,0,1.0,0.0,0.0,1,1,1,0,0,0,...,0,0,1,0,0,0,0,1,1,0
2,1,0.0,0.0,0.0,0,1,1,0,1,0,...,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8063,3,0.0,0.0,1.0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
8064,3,0.0,0.0,1.0,1,0,0,0,0,0,...,0,1,1,0,0,0,0,0,1,0
8065,3,0.0,0.0,1.0,0,0,1,0,0,0,...,0,1,1,0,0,0,0,0,0,0
8066,1,0.0,0.0,0.0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0


In [72]:
x

,Male,Yes,Yes,Doctor,Engineer,Entertainment,Executive,Healthcare,Homemaker,Lawyer,...,High,Low,30-40,40-50,50-60,60+,10+,5-10,3-6,6+
0,1,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
1,0,1,1,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
2,0,1,1,0,1,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,1,1,1,0,0,0,0,0,0,1,...,1,0,0,0,0,1,0,0,0,0
4,0,1,1,0,0,1,0,0,0,0,...,1,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2622,1,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,1,1,0
2623,0,0,1,1,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
2624,0,0,1,0,0,1,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
2625,1,1,1,0,0,0,1,0,0,0,...,1,0,0,1,0,0,0,0,1,0


In [73]:
y

0       D
1       A
2       B
3       B
4       A
       ..
2622    B
2623    A
2624    C
2625    C
2626    A
Name: Segmentation, Length: 10695, dtype: object

In [74]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2,random_state=42)

In [75]:
x_train.head()

,Male,Yes,Yes,Doctor,Engineer,Entertainment,Executive,Healthcare,Homemaker,Lawyer,...,High,Low,30-40,40-50,50-60,60+,10+,5-10,3-6,6+
3287,1,1,1,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
2513,1,1,1,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4001,1,1,1,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
58,0,0,1,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
1740,1,1,1,0,0,0,1,0,0,0,...,1,0,1,0,0,0,0,0,1,0


In [76]:
y_train.head()

3287    B
2513    A
4001    B
58      B
1740    B
Name: Segmentation, dtype: object

In [77]:
# randomforest classifier
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()

model.fit(x_train, y_train)

RandomForestClassifier()

In [78]:
print("Accuracy is", model.score(x_test, y_test)*100)

Accuracy is 45.53529686769519


In [79]:
# decision tree classifier
from sklearn.tree import DecisionTreeClassifier
model2 = DecisionTreeClassifier()
model2.fit(x_train, y_train)
print("Accuracy is", model2.score(x_test, y_test)*100)

Accuracy is 45.44179523141655


In [80]:
# logistic regression
from sklearn.linear_model import LogisticRegression
model3 = LogisticRegression()
model3.fit(x_train, y_train)
print("Accuracy is", model3.score(x_test, y_test)*100)

Accuracy is 47.68583450210379


In [81]:
#Xgboost
model4 = GradientBoostingClassifier()
model4.fit(x_train,y_train)
print("Accuracy is", model4.score(x_test, y_test)*100)

Accuracy is 49.18186068256195


In [82]:
#SVM
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC
model5 =SVC()
model5.fit(x_train,y_train)
print("Accuracy is", model5.score(x_test, y_test)*100)

Accuracy is 48.48059841047218


In [83]:
#KNN
model6 = KNeighborsClassifier()
model6.fit(x_train,y_train)
print("Accuracy is", model6.score(x_test, y_test)*100)

Accuracy is 40.065451145395045


In [84]:
train,test = train_test_split(data, test_size=0.2, random_state=42)
train.to_csv('train.csv',encoding='utf-8',index=False)
test.to_csv('test.csv',encoding='utf-8',index=False)

In [101]:
import pickle
file="model07.sav"
pickle.dump(model4, open(file,'wb'),protocol=1)

In [86]:
x_test.head()

,Male,Yes,Yes,Doctor,Engineer,Entertainment,Executive,Healthcare,Homemaker,Lawyer,...,High,Low,30-40,40-50,50-60,60+,10+,5-10,3-6,6+
3704,1,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
6750,1,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
304,0,1,1,0,0,0,0,0,0,1,...,1,0,0,0,0,1,0,0,0,0
3070,0,1,1,0,1,0,0,0,0,0,...,0,1,1,0,0,0,0,1,0,0
1597,0,1,1,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
